In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from object_detection.metrics import coco_tools
import skimage.io as io
import pylab
import os,shutil

In [2]:
annType = ['segm','bbox','keypoints']
annType = annType[1]      #specify type here
prefix = 'person_keypoints' if annType=='keypoints' else 'instances'
print ('Running demo for *%s* results.'%(annType))

Running demo for *bbox* results.


In [66]:
import os,shutil

truth_path = '/Rudy/data/tensorflow-yolov3/mAP/ground-truth/'
pred_path = '/Rudy/data/tensorflow-yolov3/mAP/predicted/'
truth_save_path='/Rudy/data/tensorflow-yolov3/mAP/COCO_truth.json'
pred_save_path='/Rudy/data/tensorflow-yolov3/mAP/COCO_pred.json'

label_map_dict = [{'id': 0, 'name': 'ZW'},
                     {'id': 1, 'name': 'ST'},
                     {'id': 2, 'name': 'HD'},
                     {'id': 3, 'name': 'YJ'},
                     {'id': 4, 'name': 'DDW'},
                     {'id': 5, 'name': 'GS'},
                     {'id': 6, 'name': 'TZ'},
                     {'id': 7, 'name': 'JJ'}]

def convert_dt(path):
    label_map_dict = {'ZW': '0','ST':'1','HD':'2','YJ':'3','DDW':'4','GS':'5','TZ':'6','JJ':'7'}
    for _,_,names in os.walk(path):
        total_list = []
        print(names)
        for name in names:
            with open (path+name,'rb') as f:
                datas = f.readlines()
                data = [line.decode().strip().split() for line in datas]
                for i in range(len(data)):
                    data_list = []
                    data[i][4],data[i][5] = int(data[i][4]) - int(data[i][2]), int(data[i][5]) - int(data[i][3])
                    data_list.append(name.split('.')[0])
                    data_list= data_list + (data[i][2:]) + data[i][1:2]
                    data_list.append(label_map_dict[data[i][0]])
                    total_list.append(data_list)
    return np.array(total_list)

def convert_gt(path):
    label_map_dict = {'ZW': '0','ST':'1','HD':'2','YJ':'3','DDW':'4','GS':'5','TZ':'6','JJ':'7'}
    for _,_,names in os.walk(path):
        total_list = []
        print(names)
        for name in names:
            with open (path+name,'rb') as f:
                datas = f.readlines()
                data = [line.decode().strip().split() for line in datas]
                for i in range(len(data)):
                    data_list = []
                    data[i][3],data[i][4] = int(data[i][3]) - int(data[i][1]), int(data[i][4]) - int(data[i][2])
                    data_list.append(name.split('.')[0])
                    data_list = data_list + (data[i][1:])
                    data_list.append(1)
                    data_list.append(label_map_dict[data[i][0]])
                    total_list.append(data_list)  
    return np.array(total_list)

def convert_OBJ_DETE_dt(path):
    label_map_dict = {'ZW': '0','ST':'1','HD':'2','YJ':'3','DDW':'4','GS':'5','TZ':'6','JJ':'7'}
    for _,_,names in os.walk(path):
        image_ids = []
        detection_boxes = []
        detection_classes = []
        detection_scores = []

        for name in names:
            image_ids.append(name.split('.')[0])
            with open (path+name,'rb') as f:
                datas = f.readlines()
                data = [line.decode().strip().split() for line in datas]
                detection_box = []
                detection_class = []
                detection_score = []
                if len(data) == 0:
                    detection_box.append([0,0,0,0])
                    detection_class.append(0)
                    detection_score.append(0)

                for i in range(len(data)):

                    detection_box.append(data[i][2:])
                    detection_class.append(label_map_dict[data[i][0]])
                    detection_score.append(data[i][1])
            detection_boxes.append(np.array(detection_box, np.float))
            detection_classes.append(np.array(detection_class, np.int32))
            detection_scores.append(np.array(detection_score, np.float))
    return image_ids,detection_boxes,detection_scores,detection_classes
def convert_OBJ_DETE_gt(path):
    label_map_dict = {'ZW': '0','ST':'1','HD':'2','YJ':'3','DDW':'4','GS':'5','TZ':'6','JJ':'7'}
    for _,_,names in os.walk(path):

        image_ids1 = []
        ground_truth_boxes = []
        ground_truth_classes = []

        for name in names:
            image_ids1.append(name.split('.')[0])
            with open (path+name,'rb') as f:
                datas = f.readlines()
                data = [line.decode().strip().split() for line in datas]
                ground_truth_box = []
                ground_truth_class = []
                if len(data) == 0:
                    ground_truth_box.append([])
                for i in range(len(data)):

                    ground_truth_box.append(data[i][1:])
                    ground_truth_class.append(label_map_dict[data[i][0]])
            ground_truth_boxes.append(np.array(ground_truth_box, np.float))
            ground_truth_classes.append(np.array(ground_truth_class, np.int32))
    return image_ids1,ground_truth_boxes,ground_truth_classes

pred_image_ids,detection_boxes,detection_scores,detection_classes = convert_OBJ_DETE_dt(pred_path)
truth_image_ids,ground_truth_boxes,ground_truth_classes=convert_OBJ_DETE_gt(truth_path)
detections_list=coco_tools.ExportDetectionsToCOCO(pred_image_ids,detection_boxes,detection_scores,detection_classes,label_map_dict)
groundtruth_dict=coco_tools.ExportGroundtruthToCOCO(truth_image_ids,ground_truth_boxes,ground_truth_classes,label_map_dict)

In [67]:
groundtruth = coco_tools.COCOWrapper(groundtruth_dict)
detections = groundtruth.LoadAnnotations(detections_list)
evaluator = coco_tools.COCOEvalWrapper(groundtruth, detections,
                                         agnostic_mode=False)
metrics,ap = evaluator.ComputeMetrics()


creating index...
index created!
INFO:tensorflow:Loading and preparing annotation results...
INFO:tensorflow:DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.07s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.236
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.432
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.195
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.055
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.048
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.258
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.229
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.273
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] 

[{'image_id': '70', 'category_id': 3, 'bbox': [0.0, 192.0, 1024.0, 87.0], 'score': 1.0}, {'image_id': '70', 'category_id': 3, 'bbox': [794.0, 275.0, 23.0, 19.0], 'score': 1.0}, {'image_id': '70', 'category_id': 3, 'bbox': [791.0, 311.0, 19.0, 28.0], 'score': 1.0}, {'image_id': '70', 'category_id': 3, 'bbox': [848.0, 302.0, 24.0, 31.0], 'score': 1.0}, {'image_id': '114', 'category_id': 5, 'bbox': [587.0, 859.0, 194.0, 111.0], 'score': 1.0}, {'image_id': '114', 'category_id': 5, 'bbox': [290.0, 289.0, 789.0, 550.0], 'score': 1.0}, {'image_id': '18', 'category_id': 5, 'bbox': [0.0, 491.0, 300.0, 456.0], 'score': 1.0}, {'image_id': '68', 'category_id': 0, 'bbox': [51.0, 726.0, 89.0, 154.0], 'score': 1.0}, {'image_id': '102', 'category_id': 0, 'bbox': [866.0, 578.0, 74.0, 99.0], 'score': 1.0}, {'image_id': '102', 'category_id': 0, 'bbox': [727.0, 654.0, 100.0, 118.0], 'score': 1.0}, {'image_id': '72', 'category_id': 0, 'bbox': [174.0, 813.0, 79.0, 78.0], 'score': 1.0}, {'image_id': '72', 'c

INFO:tensorflow:DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.07s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.627
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.945
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 1.000
 Aver

{'annotations': [{'id': 1, 'image_id': '70', 'category_id': 3, 'bbox': [0.0, 192.0, 1024.0, 87.0], 'area': 89088.0, 'iscrowd': 0, 'ignore': 0, '_ignore': 0}, {'id': 2, 'image_id': '70', 'category_id': 3, 'bbox': [794.0, 275.0, 23.0, 19.0], 'area': 437.0, 'iscrowd': 0, 'ignore': 0, '_ignore': 1}, {'id': 3, 'image_id': '70', 'category_id': 3, 'bbox': [791.0, 311.0, 19.0, 28.0], 'area': 532.0, 'iscrowd': 0, 'ignore': 0, '_ignore': 1}, {'id': 4, 'image_id': '70', 'category_id': 3, 'bbox': [848.0, 302.0, 24.0, 31.0], 'area': 744.0, 'iscrowd': 0, 'ignore': 0, '_ignore': 1}, {'id': 5, 'image_id': '114', 'category_id': 5, 'bbox': [587.0, 859.0, 194.0, 111.0], 'area': 21534.0, 'iscrowd': 0, 'ignore': 0, '_ignore': 0}, {'id': 6, 'image_id': '114', 'category_id': 5, 'bbox': [290.0, 289.0, 789.0, 550.0], 'area': 433950.0, 'iscrowd': 0, 'ignore': 0, '_ignore': 0}, {'id': 7, 'image_id': '18', 'category_id': 5, 'bbox': [0.0, 491.0, 300.0, 456.0], 'area': 136800.0, 'iscrowd': 0, 'ignore': 0, '_ignore'